In [1]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

# import tensorflow.compat.v2.feature_column as fc 
import tensorflow as tf
from tensorflow import keras


In [3]:
# Load dataset

dftrain = pd.read_csv("D:\Study\Programming_Practice\Python\Kaggle\Datasets\Titanic\TF_Train.csv")
dftest = pd.read_csv("D:\Study\Programming_Practice\Python\Kaggle\Datasets\Titanic\TF_eval.csv")
y_train = dftrain.pop("survived")
y_eval = dftest.pop("survived")

In [4]:
dftest.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,35.0,0,0,8.0500,Third,unknown,Southampton,y
1,male,54.0,0,0,51.8625,First,E,Southampton,y
2,female,58.0,0,0,26.5500,First,C,Southampton,y
3,female,55.0,0,0,16.0000,Second,unknown,Southampton,y
4,male,34.0,0,0,13.0000,Second,D,Southampton,y


In [5]:
dftrain.columns

Index(['sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck',
       'embark_town', 'alone'],
      dtype='object')

In [6]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']

numeric_coulmns = ['age', 'fare']

feature_columns = []

for features in categorical_columns:
    vocab = dftrain[features].unique() # list of unique values from features
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(features, vocab))
    
for feature in numeric_coulmns:
    feature_columns.append(tf.feature_column.numeric_column(feature, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [7]:
dftrain["sex"].unique()

array(['male', 'female'], dtype=object)

In [44]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle= True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_input_fnc = make_input_fn(dftrain, y_train)
eval_input_fnc = make_input_fn(dftest, y_eval, num_epochs= 1, shuffle= False)


In [46]:
linear_est = tf.estimator.LinearClassifier(feature_columns= feature_columns)
linear_est.train(train_input_fnc) # training
result = linear_est.evaluate(eval_input_fnc)

clear_output()
print(result["accuracy"])

0.75


In [54]:
result = list(linear_est.predict(eval_input_fnc))
print(dftest.loc[2])
print(result[2] ['probabilities'][1]) # servive

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\mdmuz\AppData\Local\Temp\tmpdr5stnmu\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                        female
age                          58.0
n_siblings_spouses              0
parch                           0
fare                        26.55
class                       First
deck                            C
embark_town           Southampton
alone                           y
Name: 2, dtype: object
0.69045323
